# NMEA Parsing, in Python

## Full Sample, RMC Parsing, from scratch. 
### Reading the serial port to displaying readable data

#### Reading the Serial port
Make sure you have `pyserial` installed
```
$ pip3 install pyserial
```

In [ ]:
import serial

> *Important*: Modify the port name below, so it matches what the Jupyter server can see.

In [ ]:
port_name = "/dev/tty.usbmodem14101"
# port_name = "/dev/ttyS80"
baud_rate = 4800
port = serial.Serial(port_name, baudrate=baud_rate, timeout=3.0)

Let us try to read the serial port
> Note: We limit the read to 200 characters

In [ ]:
nb_read = 0
while True:
    try:
        ch = port.read()
        print("{:02x} ".format(ord(ch)), end='', flush=True)
        nb_read += 1
        if nb_read > 200:
            break
    except KeyboardInterrupt as ki:
        break
    except Exception as ex:
        print("Argh! {}".format(ex))

print("\n\n\t\tDone reading, exiting.")
port.close()

print("Bye.")

We can see that data are being read from the Serial port. Now let us try to make some sense out of them.

We know that NMEA Sentences begin with a `$` sign, and end with `\r\n` (Carriage Return, New Line).
Let us try to extract the NMEA Sentences from the continuous serial flow.
We will stop when 20 sentences are read.

In [ ]:
port = serial.Serial(port_name, baudrate=baud_rate, timeout=3.0)
previous_char = ''
nb_sentences = 0
rv = []
#
while True:
    try:
        ch = port.read()
        if ord(ch) == ord('$'):
            print("\nNew sentence?")
        print("{:02x} ".format(ord(ch)), end='', flush=True)
        rv.append(ch)
        if ord(ch) == 0x0A and ord(previous_char) == 0x0D:
            string = "".join(map(bytes.decode, rv))
            print("\nEnd of sentence {}".format(string))
            nb_sentences += 1
            rv = []
            if nb_sentences > 20:
                break
        previous_char = ch
    except KeyboardInterrupt as ki:
        break
    except Exception as ex:
            print("Argh! {}".format(ex))
            nb_sentence += 1  # Not to end up in infinite loop...
            
print("\n\n\t\tDone reading, exiting.")
port.close()

print("Bye.")

So, now we can extract NMEA sentences from the Serial flow. Let's move on, and try to extract the data conveyed by those sentences.


#### Validation. Example: RMC sentence
Set the data string to validate

In [22]:
rmc_data = "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n"

Will keep going as long as good

In [23]:
valid = True

Validate start of the string

In [24]:
if rmc_data[0] != '$':
    valid = False
    print("String does not begin with '$', not valid")
else:
    print("Start of String OK, moving on.")

Start of String OK, moving on.


Validate the end

In [25]:
if valid:
    if rmc_data[-2:] != "\r\n":
        valid = False
        print("Bad string termination")
    else:
        print("String termination OK, moving on.")

String termination OK, moving on.


Now, the checksum

In [26]:
if valid:
    string_to_validate = rmc_data[1:-5]
    checksum = rmc_data[-4:-2]
    print("Data to validate: {} against {}".format(string_to_validate, checksum))

Data to validate: GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A against 7C


In [27]:
if valid:
    cs = 0
    char_array = list(string_to_validate)
    for c in range(len(string_to_validate)):
        cs = cs ^ ord(char_array[c])
        print ("Char {} (0x{:02x} 0b{}) -> CheckSum now 0x{:02x} 0b{}".format(
            char_array[c], 
            ord(char_array[c]), 
            str(bin(ord(char_array[c])))[2:].rjust(8, '0'), 
            cs,
            str(bin(cs))[2:].rjust(8, '0')))
    original_cs = int(checksum, 16)
    if original_cs != cs:
        valid = False
        print("Invalid Checksum: Found {:02x}, expected {:02x}".format(cs, original_cs))
    else:
        print("Checksum OK, moving on")
    

Char G (0x47 0b01000111) -> CheckSum now 0x47 0b01000111
Char P (0x50 0b01010000) -> CheckSum now 0x17 0b00010111
Char R (0x52 0b01010010) -> CheckSum now 0x45 0b01000101
Char M (0x4d 0b01001101) -> CheckSum now 0x08 0b00001000
Char C (0x43 0b01000011) -> CheckSum now 0x4b 0b01001011
Char , (0x2c 0b00101100) -> CheckSum now 0x67 0b01100111
Char 1 (0x31 0b00110001) -> CheckSum now 0x56 0b01010110
Char 8 (0x38 0b00111000) -> CheckSum now 0x6e 0b01101110
Char 3 (0x33 0b00110011) -> CheckSum now 0x5d 0b01011101
Char 3 (0x33 0b00110011) -> CheckSum now 0x6e 0b01101110
Char 3 (0x33 0b00110011) -> CheckSum now 0x5d 0b01011101
Char 3 (0x33 0b00110011) -> CheckSum now 0x6e 0b01101110
Char . (0x2e 0b00101110) -> CheckSum now 0x40 0b01000000
Char 0 (0x30 0b00110000) -> CheckSum now 0x70 0b01110000
Char 0 (0x30 0b00110000) -> CheckSum now 0x40 0b01000000
Char 0 (0x30 0b00110000) -> CheckSum now 0x70 0b01110000
Char , (0x2c 0b00101100) -> CheckSum now 0x5c 0b01011100
Char A (0x41 0b01000001) -> Che

### Validations OK
Now splitting the data

In [28]:
if valid:
    members = string_to_validate.split(',')
    print("We have {} members:".format(len(members)))
    for item in members:
        print(item if len(item) > 0 else '-')

We have 13 members:
GPRMC
183333.000
A
4047.7034
N
07247.9938
W
0.66
196.21
150912
-
-
A


In [29]:
if valid:
    if len(members[0]) != 5:
        print("Bad length for sentence prefix and ID")
    else:
        device_prefix = members[0][0:2]
        sentence_id = members[0][-3:]
        print("Device Prefix is {}, Sentence ID is {}".format(device_prefix, sentence_id))

Device Prefix is GP, Sentence ID is RMC


#### Quick utility: `Decimal to Sexagesimal` and vice-versa

In [30]:
import math

NS = 0
EW = 1

def dec_to_sex(value, type):
    abs_val = abs(value)  # (-value) if (value < 0) else value
    int_value = math.floor(abs_val)
    i = int(int_value)
    dec = abs_val - int_value
    dec *= 60
    sign = "N"
    if type == NS:
        if value < 0:
            sign = "S"
    else:
        if value < 0:
            sign = "W"
        else:
            sign = "E"
    formatted = "{} {}\272{:0.2f}'".format(sign, i, dec)
    return formatted


def sex_to_dec(deg_str, min_str):
    """
    Sexagesimal to decimal
    :param deg_str: degrees value (as string containing an int) like '12'
    :param min_str: minutes value (as a string containing a float) like '45.00'
    :return: decimal value, like 12.75 here.
    """
    try:
        degrees = float(deg_str)
        minutes = float(min_str)
        minutes *= (10.0 / 6.0)
        ret = degrees + minutes / 100.0
        return ret
    except ValueError:
        raise Exception("Bad numbers [{}] [{}]".format(deg_str, min_str))


In [31]:
sample_one = -122.5069175      
sample_two = 37.748837
print("{} becomes {}".format(sample_one, dec_to_sex(sample_one, EW)))
print("{} becomes {}".format(sample_two, dec_to_sex(sample_two, NS)))

-122.5069175 becomes W 122º30.42'
37.748837 becomes N 37º44.93'


In [32]:
deg_1 = '122'
min_1 = '30.42'
print("{} becomes {}".format(deg_1 + '\272' + min_1 + "'", sex_to_dec(deg_1, min_1)))

122º30.42' becomes 122.507


### A note about Latitude and Longitude
In the `RMC` String above, the position is represented by
```
 4047.7034,N,07247.9938,W
```
- The latitude `4047.7034` must be read `40` degrees and `47.7034` minutes.
- The longitude `07247.9938` means `72` degrees and `47.9938` minutes.

In the `RMC` case:
- Element with index `3` holds the latitude's value (absolute)
- Element with index `4` holds the latitude's sign (`N` or `S`)
- Element with index `5` holds the longitude's value (absolute)
- Element with index `6` holds the longitude's sign (`E` or `W`)

```
                                                                    12
  0      1      2 3        4 5         6 7     8     9      10    11
  $GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W,A*6A
         |      | |        | |         | |     |     |      |     | |
         |      | |        | |         | |     |     |      |     | Type: A=autonomous,
         |      | |        | |         | |     |     |      |     |       D=differential,
         |      | |        | |         | |     |     |      |     |       E=Estimated,
         |      | |        | |         | |     |     |      |     |       N=not valid,
         |      | |        | |         | |     |     |      |     |       S=Simulator
         |      | |        | |         | |     |     |      |     Variation sign
         |      | |        | |         | |     |     |      Variation value
         |      | |        | |         | |     |     Date DDMMYY
         |      | |        | |         | |     COG
         |      | |        | |         | SOG
         |      | |        | |         Longitude Sign
         |      | |        | Longitude Value
         |      | |        Latitude Sign
         |      | Latitude value
         |      Active or Void
         UTC
```
Let's re-display the array of data with the labels:

In [33]:
rmc_labels = [
    "Key",
    "UTC",
    "Act",
    "Lat Value",
    "Lat Sign",
    "Long Value",
    "Long Sign",
    "SOG",
    "COG",
    "UTC Date",
    "Variation value",
    "Variation Sign",
    "Type"
]
for i in range(len(members)):
    try:
        val = members[i]
    except IndexError:
        val = '-'
    print("{}: {}".format(rmc_labels[i].rjust(20, ' '), val if len(val) > 0 else '-'))


                 Key: GPRMC
                 UTC: 183333.000
                 Act: A
           Lat Value: 4047.7034
            Lat Sign: N
          Long Value: 07247.9938
           Long Sign: W
                 SOG: 0.66
                 COG: 196.21
            UTC Date: 150912
     Variation value: -
      Variation Sign: -
                Type: A


Now, extract position:

In [34]:
# Extract Latitude
abs_lat = float(members[3]) / 100
lat_deg = int(abs_lat)
lat_min = (abs_lat - lat_deg) * 100
print("Abs Lat: {}, Lat Deg: {}, Lat Min: {} ".format(abs_lat, lat_deg, lat_min))
decimal_lat = sex_to_dec(str(lat_deg), str(lat_min))
decimal_lat *= (1 if members[4] == 'N' else -1)
print("Latitude {}".format(decimal_lat))

Abs Lat: 40.477033999999996, Lat Deg: 40, Lat Min: 47.70339999999962 
Latitude 40.79505666666666


In [35]:
# Extract Longitude
abs_lng = float(members[5]) / 100
lng_deg = int(abs_lng)
lng_min = (abs_lng - lng_deg) * 100
print("Abs Lng: {}, Lng Deg: {}, Lng Min: {} ".format(abs_lng, lng_deg, lng_min))
decimal_lng = sex_to_dec(str(lng_deg), str(lng_min))
decimal_lng *= (1 if members[6] == 'E' else -1)
print("Longitude {}".format(decimal_lng))

Abs Lng: 72.479938, Lng Deg: 72, Lng Min: 47.99380000000042 
Longitude -72.79989666666667


### Using the `nmea_parser.py`

In [ ]:
import json
import nmea_parser as NMEAParser

In [ ]:
samples = [
    "$IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D\r\n",
    "$IIMWV,088,T,14.34,N,A*27\r\n",
    "$IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX\r\n",
    "$IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX\r\n",
    "$GPTXT,01,01,02,u-blox ag - www.u-blox.com*50\r\n",
    "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n",
    "$IIGLL,3739.854,N,12222.812,W,014003,A,A*49\r\n"
]

In [ ]:
for sentence in samples:
    print("Parsing {}".format(sentence))
    try:
        nmea_obj = NMEAParser.parse_nmea_sentence(sentence)
        try:
            print('Parsed Object: {}'.format(json.dumps(nmea_obj, indent=2)))
        except TypeError as type_error:
            print('TypeError: {}'.format(type_error))
            print('Parsed Object (raw): {}'.format(nmea_obj))
    except Exception as ex:
        print("Ooops! {}, {}".format(type(ex), ex))
    print("----------------------------------")
